# Lab: Fine tuning
---

In this demo we are going to fine-tune pretrained model. This demo is based on open-source materials from [huggingface.co docs](https://huggingface.co/docs/transformers/training#train-with-pytorch-trainer).

## Setting up the environment

For this demo we are going to need the following packages:

In [ ]:
! pip install transformers datasets torch evaluate scikit-learn

## Loading the dataset

We are going to load the Yelp Reviews dataset which comes enabled 

In [7]:
from datasets import load_dataset

dataset = load_dataset("yelp_review_full")

dataset["train"][0]

Found cached dataset yelp_review_full (/Users/ajdinmujezinovic/.cache/huggingface/datasets/yelp_review_full/yelp_review_full/1.0.0/e8e18e19d7be9e75642fc66b198abadb116f73599ec89a69ba5dd8d1e57ba0bf)
100%|██████████| 2/2 [00:00<00:00, 186.03it/s]


{'label': 4,
 'text': "dr. goldberg offers everything i look for in a general practitioner.  he's nice and easy to talk to without being patronizing; he's always on time in seeing his patients; he's affiliated with a top-notch hospital (nyu) which my parents have explained to me is very important in case something happens and you need surgery; and you can get referrals to see specialists without having to see him first.  really, what more do you need?  i'm sitting here trying to think of any complaints i have about him, but i'm really drawing a blank."}

A tokenizer is required to handle text, along with a padding and truncation approach to accommodate varying sequence lengths. To process your entire dataset in a single step, you can employ the `map` method from  Datasets and apply a preprocessing function over it.

In [8]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")


def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)


tokenized_datasets = dataset.map(tokenize_function, batched=True)

Loading cached processed dataset at /Users/ajdinmujezinovic/.cache/huggingface/datasets/yelp_review_full/yelp_review_full/1.0.0/e8e18e19d7be9e75642fc66b198abadb116f73599ec89a69ba5dd8d1e57ba0bf/cache-a672216cdfb09c9b.arrow
Loading cached processed dataset at /Users/ajdinmujezinovic/.cache/huggingface/datasets/yelp_review_full/yelp_review_full/1.0.0/e8e18e19d7be9e75642fc66b198abadb116f73599ec89a69ba5dd8d1e57ba0bf/cache-ae71c631b0b6a605.arrow


To reduce fine-tuning, we are going to use smaller subset of the data.

In [19]:
small_train_dataset = tokenized_datasets["train"].shuffle(seed=42).select(range(10))
small_eval_dataset = tokenized_datasets["test"].shuffle(seed=42).select(range(10))

Loading cached shuffled indices for dataset at /Users/ajdinmujezinovic/.cache/huggingface/datasets/yelp_review_full/yelp_review_full/1.0.0/e8e18e19d7be9e75642fc66b198abadb116f73599ec89a69ba5dd8d1e57ba0bf/cache-9edbabb881154e45.arrow
Loading cached shuffled indices for dataset at /Users/ajdinmujezinovic/.cache/huggingface/datasets/yelp_review_full/yelp_review_full/1.0.0/e8e18e19d7be9e75642fc66b198abadb116f73599ec89a69ba5dd8d1e57ba0bf/cache-51e4392d834261d1.arrow


The 🤗 Transformers library includes a specialized Trainer class that is optimized for training 🤗 Transformers models, eliminating the need to manually create a training loop. The Trainer API provides a variety of training options and features, such as logging, gradient accumulation, and mixed precision.

To begin, load your model and specify the number of expected labels. As per the Yelp Review dataset card, there are five labels in total.

In [20]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained("bert-base-cased", num_labels=5)

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at b

Subsequently, instantiate a TrainingArguments class that comprises various hyperparameters that can be adjusted, along with flags for enabling diverse training options. For this tutorial, you can begin with the default training hyperparameters, although you are encouraged to experiment with them to determine your ideal settings.

Designate the location where you wish to save the checkpoints generated during training.

In [21]:
from transformers import TrainingArguments

training_args = TrainingArguments(output_dir='/tmp/test_trainer')

During training, the Trainer class does not automatically evaluate model performance. Therefore, you must provide a function to calculate and report metrics to the Trainer. You can employ the 🤗 Evaluate library to load a simple accuracy function using the evaluate.load function (check out this quicktour for additional information).





In [22]:
import numpy as np
import evaluate

metric = evaluate.load("accuracy")

Invoke the compute method on metric to determine the accuracy of your predictions. However, before feeding your predictions to compute, you must convert them to logits (recall that all 🤗 Transformers models generate logits).

In [23]:
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

To monitor evaluation metrics during the process of fine-tuning, we can specify `evaluation_strategy` parameter.

In [24]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(
    output_dir="/tmp/test_trainer", evaluation_strategy="epoch")

Finally, we can create `Trainer` object with all the necessary parameters:

In [25]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=small_train_dataset,
    eval_dataset=small_eval_dataset,
    compute_metrics=compute_metrics,
)

To initiate fine-tuning, we call `train()`:

In [27]:
# This step runs 50mins+ so we are not going to run it now.
# trainer.train()